# 추천 시스템

- **콘텐츠 기반 필터링**(Content based filtering)

- **협업 필터링**(Collaborative Filtering)  
  1. **최근접 이웃**(Nearest Neighbor) 협업 필터링 (= **메모리(Memory)** 협업 필터링)
  2. **잠재 요인**(Latent Factor) 협업 필터링
<br>
<br>
**목표**  : <span style="color: #2D3748; background-color:#fff5b1;">**사용자가 아직 평가하지 않은 아이템을 예측 평가**</span>하는 것  
<br>
**특징**  
  1. <span style="color: #2D3748; background-color:#fff5b1;">**사용자-아이템 평점 행렬 데이터에만 의지**</span>
  2. 행은 개별 사용자, 열은 개별 아이템, 해당하는 값이 평점을 나타내는 형태가 되어야 함
  3. 데이터가 레코드 레벨 형태인 경우, 판다스의 pivot_table()과 같은 함수를 이용해 사용자-아이템 평점 행렬 형태로 변경해주어야 함  
  
<br>
- 희소 행렬(sparse matrix) 특성
- 다차원 행렬

## 콘텐츠 기반 필터링

사용자가 특정한 아이템을 매우 선호하는 경우, 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템을 추천하는 방식  
**제품(상품)에 초점**을 맞춤  
ex) 영화

## 최근접 이웃 협업 필터링

친구들에게 물어보는 것과 유사한 방식  
(사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은) **사용자 행동 양식(User Behavior)을 기반**으로 추천을 수행  
  
  - "메모리 협업 필터링" 이라고도 함
  - **사용자 기반**(User-User) : 당신과 비슷한 고객(사용자 간)
  - **아이템 기반**(Item-Item) : 이 상품을 선택한 다른 고객(<span style="color: #2D3748; background-color:#fff5b1;">아이템이 가지는 속성과는 상관없음</span>)

## 잠재 요인 협업 필터링

**사용자-아이템 행렬 데이터**만을 이용해 **"행렬 분해 기법"을 사용**하여 **'잠재 요인'을 끄집어 내는 것**

------------

## 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### 장르 속성을 이용한 영화 콘텐츠 기반 필터링

- 콘텐츠 기반 필터링 : 상품/서비스 간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠를 기반으로 하는 방식

### 데이터 로딩 및 가공

In [2]:
import pandas as pd

In [30]:
movies = pd.read_csv('./data/tmdb_5000_movies.csv')
movies[:2]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [31]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [32]:
movies['genres']

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                   []
4802                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4803, dtype: object

리스트로 묶여 있고, 딕셔너리 형태로 보이지만, 판다스 셀 안에 리스트나 딕셔너리 객체가 들어갈 수 없음  
info 보면 그냥 object 임을 알 수 있음  

In [33]:
movies['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [34]:
movies['genres'][0][1]

'{'

문자열을 인덱싱하는 것을 알 수 있음

In [35]:
# 문자열을 파이썬의 객체로 변환하는 함수

In [36]:
from ast import literal_eval

In [37]:
type(literal_eval("{'id' : 28, 'name' : 'Action'}"))

dict

In [38]:
literal_eval(movies['genres'][0][0])

SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [39]:
movies_df = movies.iloc[:, [3, 17, 1, 18, 19, 8, 4, 7]]
movies_df[:2]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."


In [41]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [42]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [48]:
for i in movies_df['genres'][0]:
    print(i['name'])

Action
Adventure
Fantasy
Science Fiction


In [49]:
[i['name'] for i in movies_df['genres'][0]]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [50]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x ])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x ])

In [52]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


In [54]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 문장열로 변환해주기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))  # 괄호없이 ' ' 이렇게 해도 됨

count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

genre_mat.shape

(4803, 276)

In [57]:
# 코사인 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [62]:
print(cosine_similarity(genre_mat[0], genre_mat)) # 첫 번째 장르와 모든 장르의 유사도 측정 방법

cosine_similarity(genre_mat[0], genre_mat[1]) # 첫 번째 장르와 두 번재 장르의 유사도 측정 방법

[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


array([[0.59628479]])

In [74]:
import numpy as np
genre_sim_sorted_ind = np.argsort(genre_sim)[:, ::-1]  # 유사도 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값 얻기

=> 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고, 가장 유사도가 낮은 레코드는 2401번 레코드임  
이 위치 인덱스를 이용해 언지든지 특정 레코드와 코사인 유사도가 높은 다른 레코드를 추출할 수 있음

In [66]:
a = np.array([10, 13, 14, 11])
np.sort(a)

array([10, 11, 13, 14])

In [67]:
np.argsort(a)

array([0, 3, 1, 2], dtype=int64)

In [68]:
np.argsort(a)[::-1]

array([2, 1, 3, 0], dtype=int64)

In [70]:
a[[2, 1, 3, 0]]

array([14, 13, 11, 10])

### 장르 콘텐츠 필터링을 이용한 영화 추천

장르 유사도에 따라 영화를 추천하는 함수 생성하기

In [71]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    
    # 인자로 입력된 movies_df Dataframe에서 title 컬럼이 입력된 title_name 값인 dataframe 추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 dataframe의 index 객체를 ndarray로 반환
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, : (top_n)]
    
    # 추출된 top_n index 출력. top_n index는 2차원 데이터임
    # datafram에서 index로 사용하기 위해서 1차원 array로 변경\
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [75]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


#### 가중 평점 공식

평가 횟수에 대한 **가중치가 부여**된 평점 방식

가중 평점(Weighted Rating) = ($\frac{v}{v+m}$) * R + ($\frac{m}{v+m}$) * C

In [77]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print(C, m)

6.092171559442011 370.1999999999998


In [78]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return (v / (v + m)) * R + (m / (v + m)) * C

In [87]:
movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis = 1)

In [88]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


----------

## Surprise - 파이썬 추천 시스템 패키지

In [103]:
conda install scikit-learn

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Min\anaconda3

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fftw-3.3.9                 |       h2bbff1b_1         672 KB
    icc_rt-2022.1.0            |       h6049295_2         6.5 MB
    scikit-learn-1.1.2         |   py39hd77b12b_0         5.5 MB
    scipy-1.9.1                |   py39he11b74f_0        15.7 MB
    ------------------------------------------------------------
                                           Total:        28.4 MB

The following NEW packages will be INSTALLED:

  fftw               pkgs/main/win-64::fftw-3.3.9-h2bbff1b_1 None
  icc_rt             pkgs/main/win-64::icc_rt-2022.1.0-h6049295_2 None
  scikit-learn       pkgs/main/win-64::scikit-learn-1.1.2-py39hd77b12b_0 None
  scipy              pkgs/main/win-64::scipy-1.9

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Min/.surprise_data/ml-100k


In [4]:
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

In [5]:
algo = SVD(random_state = 0)
algo.fit(trainset)

In [12]:
pred = algo.test(testset)
pred

[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False}),
 Prediction(uid='219', iid='82', r_ui=1.0, est=4.1755161590745145, details={'was_impossible': False}),
 Prediction(uid='279', iid='571', r_ui=4.0, est=2.4179600970240775, details={'was_impossible': False}),
 Prediction(uid='429', iid='568', r_ui=3.0, est=3.062893143794347, details={'was_impossible': False}),
 Prediction(uid='456', iid='100', r_ui=3.0, est=4.51573882377166, details={'was_impossible': False}),
 Prediction(uid='249', iid='23', r_ui=4.0, est=4.76990278783951, detai

In [13]:
pred[0]

Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False})

In [14]:
[  (pred.uid, pred.iid, pred.est) for pred in preds[:3]  ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [15]:
preds = algo.predict(str(196), str(302))

In [16]:
preds

Prediction(uid='196', iid='302', r_ui=None, est=4.494386477040736, details={'was_impossible': False})

In [18]:
accuracy.rmse(algo.test(testset))

RMSE: 0.9467


0.9466860806937948

### surprise를 이용한 개인화 영화 추천 시스템

In [28]:
from surprise.model_selection import cross_validate
import pandas as pd
from surprise import Reader
from surprise import SVD


ratings = pd.read_csv('./data/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

In [30]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors = 50, random_state = 0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'